In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

DIR = r"/home/cz/Dropbox/Summer RA Model EP/Parameters/"

def pv(name, mat):
    print("{}: {}".format(name.rjust(10), mat.shape))

In [2]:
from parameters import read_params
from scipy.special import gamma
from copy import copy

def myGamma(zeta):
    return gamma(1-1/zeta)

def psi(r):
    n = param.eta
    ret = 1 + (param.beta**(1/(1-n))) * ((1+r)**(n/(1-n)))
    return 1/ret

def fix_params(param):
    print('----'*10)
    print('Reshaped:')
    param_c = copy(param)
    
    # reset 
    skills, wages = param.skill1880, param.wage1880
    Q_A, Q_NA = param.Q_A, param.Q_NA

    Z, vA, Capital, phi = param.Z, param.vA, param.Capital, param.phi

    tau, amenities = param.tau, param.amenities

    # reshape
    skills = np.expand_dims(skills, 2).transpose((2,1,0))
    param_c.skills = skills
    pv("skills", skills)

    wages = np.expand_dims(wages, 2).transpose((2,1,0))
    param_c.wages = wages
    pv("wages", wages)

    Q = np.stack((Q_A, Q_NA), axis=1).transpose((2,1,0))
    param_c.Q = Q
    pv("Q", Q)

    amenities = amenities.transpose((1,0))
    param_c.amenities = amenities
    pv("amenities", amenities)

    vA = vA.flatten()
    param_c.vA = vA
    
    realvartheta_A= np.array([vA, 1-vA]).T
    realvartheta_A= np.expand_dims(realvartheta_A, 2)
    param_c.vartheta = realvartheta_A
    pv("vA", realvartheta_A)
    
    vAt = realvartheta_A[1]
    param_c.vAt = vAt
    pv("vA (1880)", vAt)

    Capital = Capital.flatten()
    param_c.Capital = Capital
    pv("Capital", Capital)

    pv("tau", tau)
    
    return param_c


In [3]:
raw = read_params()
param = fix_params(raw)

Found 13 parameters
param.beta = 0.28818
param.q = 0.67565
param.mu = 3.4112
param.zeta = 1.62
param.alpha = 0.05
param.sigma = 4
param.psi = 0.76551
param.delta = 0.91803
param.eta = 0.35
param.gamma = 0.35055
param.nu = 0.0187
param.kappa = 0.42
param.r = 2.8866

 skill1880: (645, 2)
  wage1880: (645, 2)
       Q_A: (645, 5)
      Q_NA: (645, 5)
         Z: (5, 2)
        vA: (5, 1)
   Capital: (5, 1)
       phi: (4, 1)
       tau: (645, 645)
 amenities: (645, 5)
----------------------------------------
Reshaped:
    skills: (1, 2, 645)
     wages: (1, 2, 645)
         Q: (5, 2, 645)
 amenities: (5, 645)
        vA: (5, 2, 1)
 vA (1880): (2, 1)
   Capital: (5,)
       tau: (645, 645)


In [4]:
# define some constants for indexing
A, NA = 0,1
L, H = 0,1
R,S,T = 2,1,0

# Remember that Psi is indexed like Psi[L,A]
Psi = np.array([[1,1],
                [param.q, param.mu * param.q]])

In [5]:
       
def computePi(wages):
    """
    Parameters:
        wages - shape [time, Sectors, Regions]
        Q - shape [time, Sectors, Regions]
    
    Returns:
        pi has shape (1, 2, 645)
    """
    # Q is already raised**(sigma - 1)
    pi = param.Q[t+1] * wages**((1-param.alpha)*(1-param.sigma)) # use 1910 Q's
    
    # sum across regions
    normalize = np.sum(pi, axis=R)
    
    # normalize has shape (1,2), so we make it have shape (1,2,1)
    normalize = np.expand_dims(normalize, axis=2)
    pi = pi / normalize
    
    return pi

def computeTheta(wages):
    """
    196 µs ± 1.95 µs per loop (mean ± std. dev. of 7 runs)
    """
    Theta = np.matmul(Psi, wages**(param.zeta)) ** (1/param.zeta)
    Theta_l = Theta[0,L]
    Theta_h = Theta[0,H]
    return Theta_h, Theta_l

def computeLrt(skills):
    """
    Expects:
        skills[1,2,645]
    
    Returns:
        const_lambda - Lambda, 1-Lambda. This is constant throughout time
        totals - L. (portion of population in each region)
        skill_shares - share of high/low skill in each region. lambda_rt and 1-lambda_rt
        
    """
    const_lamda = np.sum(skills, axis=R)
    sums = np.sum(skills, axis=S)
    skill_shares = (skills/sums)
    return const_lamda, sums, skill_shares


def computeGDP(skills, wages):
    Theta_h, Theta_l = computeTheta(wages)
    
    constant= 1/(1-param.alpha) * myGamma(param.zeta)
    GDP = constant * np.sum((skills[0,H] * Theta_h + skills[0,L] * Theta_l))
    
    return GDP

def priceNormalize(wages):
    """
    Sets the price of the non-agricultural good to 1
    
    Parameters:
        Q_rst
        Z_st
        param.alpha
        R_t
        
    Returns:
        normalized wages
    """
    x = param.Q[t].transpose((1,0)) * (1/param.Z[t] * (wages[0].transpose((1,0)) / (1-param.alpha))**(1-param.alpha)* (param.R_t/param.alpha)**param.alpha)**(1-param.sigma)
    x = x.transpose((1,0))
    prices = np.sum(x, axis=1)**(1/(1-param.sigma))
    normprices = prices/prices[NA]

    wages = wages * ((1/prices[NA])**(1/(1-param.alpha)))
    return wages

In [6]:
def loopy(skills, wages, log=True):

    lr = 0.03

    # these are fixed
    const_lamda, sums, skill_shares = computeLrt(skills)

    previous_newWages = wages
    for i in range(300):
        # normalize wages BY THE PRICE OF THE AGRICULTURAL GOOD

        wages = priceNormalize(wages)

        # compute the update
        pi = computePi(wages) # (1 by 2 by 645)
        Theta_h, Theta_l = computeTheta(wages) # (645,)
        GDP = computeGDP(skills, wages)
        revenue = (1-param.alpha) * pi * param.vAt * GDP
        GDP

        #  labor market clearing conditions

        # A
        s_hA = Psi[H,A] * (wages[0,A] / Theta_h)**(param.zeta)
        s_lA = Psi[L,A] * (wages[0,A] / Theta_l)**(param.zeta)

        # NA
        s_hNA = Psi[H,NA] * (wages[0,NA] / Theta_h)**(param.zeta)
        s_lNA = Psi[L,NA] * (wages[0,NA] / Theta_l)**(param.zeta)

        # w*(H_highskill + H_lowskill) == left hand side (for a given sector)
        zeta = param.zeta
        H_hA = skills[0,H] * (Psi[H,A]**(1/zeta)) * (s_hA**(1-1/zeta))
        H_hNA = skills[0,H] * (Psi[H,NA]**(1/zeta)) * (s_hNA**(1-1/zeta))
        H_lA = skills[0,L] * (Psi[L,A]**(1/zeta)) * (s_lA**(1-1/zeta))
        H_lNA = skills[0,L] * (Psi[L,NA]**(1/zeta)) * (s_lNA**(1-1/zeta))

        x_A = myGamma(zeta) * (H_hA + H_lA)
        x_NA= myGamma(zeta) * (H_hNA + H_lNA)

        newWages = revenue / np.vstack([x_A, x_NA])
        
        loss = np.sum((newWages - previous_newWages)**2)
        previous_newWages = newWages

        wages = lr * newWages + (1-lr) * wages

        if loss < 1e-7:
            if log:
                print('w-- #{}: {}'.format(i, loss))
            break

        if i%20 == 0 and log:
            print(i, np.sum(newWages), np.sum(wages), loss)
            
    return wages

In [11]:
t = 0

def computeMovingProbs(wages):
    
    theta_h, theta_l = computeTheta(wages)
    Theta = np.expand_dims(np.vstack([theta_l, theta_h]), 0)
    expectedUtility = myGamma(param.zeta)*Theta
    
    constant = myGamma(param.eta/param.zeta)/param.eta * psi(param.r)**(param.eta-1)
    
    # since the interest rate is a constant, we can take psi(r_{t+1}) to be a constant too
    W_rt = constant * expectedUtility**param.eta + param.amenities[t+1] # amenities in 1910

    # tau is indexed like j x r
    stuff = np.expand_dims(W_rt, 2) - param.tau
    rho = np.exp(1/param.kappa * stuff)
    
    # and we normalize
    sums = np.sum(rho, axis=3) # across the R's, not the j's
    sums = np.expand_dims(sums, axis=3)

    movingProbs = rho/sums
    return movingProbs


def newLocations(skills, wages):
    """
    updates the new locations
    """
    movingProbs = computeMovingProbs(wages)
    oldL = np.expand_dims(skills, axis=3)
    new = np.sum(movingProbs * oldL, axis=2)
    
    return new

def solveSpace(skills, wages, log=10):
    
    lr = 0.2
    losses = []

    # loop
    for i in range(500):

        # find a fixed point wage, given location
        wages = loopy(skills, wages, log=False)

        # find the new locations, given the wage
        newSkills = newLocations(skills, wages)

        loss = np.sum((newSkills - skills)**2)
        losses.append(loss)

        skills = lr * newSkills + (1-lr)*skills

        if loss < 1e-6:
            print('L---- #{}: {}'.format(i, loss))
            break

        if i % log == 0:
            print('L---- #{}: {}'.format(i, loss))
    
    return skills, wages

In [12]:
h_wages = [param.wages]
h_skills = [param.skills]
param.R_t = 3.63

import time

for i in range(4):
    start = time.time()
    t = i
    
    # init guesses
    guess_wages  = np.ones_like(param.wages)
    guess_skills = param.skills
    
    newSkills, newWages = solveSpace(guess_skills, guess_wages, log=100)
    h_wages.append(newWages)
    h_skills.append(newSkills)
    print('{} took {} seconds'.format(1910+30*i, time.time() - start))

L---- #0: 0.00039380222810324724
L---- #100: 1.786987824524661e-05
L---- #200: 1.1428210305023524e-05
L---- #300: 6.778459046263705e-06
L---- #400: 3.8923751453358385e-06
1910 took 18.96175503730774 seconds
L---- #0: 0.0006772284092966241
L---- #100: 2.4092379184263553e-05
L---- #200: 1.3905008541588025e-05
L---- #300: 9.887873290374224e-06
L---- #400: 7.2313285302634896e-06
1940 took 20.77356767654419 seconds
L---- #0: 0.0013916079269217204
L---- #100: 5.756251066947969e-05
L---- #200: 1.896232045380922e-05
L---- #300: 7.919562891154113e-06
L---- #400: 3.7500586149772654e-06
1970 took 26.58291482925415 seconds
L---- #0: 0.001210166262720822
L---- #100: 7.730062779381391e-06
L---- #200: 2.699620341014682e-06
L---- #300: 1.3229886293947654e-06
L---- #344: 9.948521878674302e-07
2000 took 20.33562660217285 seconds


In [14]:
for i in range(4):
    name = 'records/{}'.format('{}-'.format(1910+30*i))
    np.save(name + 'wages', h_wages[i+1])
    np.save(name + 'skills', h_skills[i+1])
    print(name)

records/1910-
records/1940-
records/1970-
records/2000-
